In [1]:
from sklearn.metrics import mean_absolute_percentage_error as mape
from all_functions import *
from darts import TimeSeries
import os
from darts.models import StatsForecastAutoETS
from darts.models import StatsForecastAutoARIMA
import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
def percorrer_csv(diretorios, save_dir='./', horizon=5):
    cont = 0
    
    resultado_file = save_dir+'/result_ets.csv'
    train = pd.Series()
    for diretorio in diretorios.split():
        if os.path.isdir(diretorio):
            for root, _, arquivos in os.walk(diretorio):
                for arquivo in arquivos:
                    if arquivo.endswith('.csv'):
                        caminho_arquivo = os.path.join(root, arquivo)
                        try:
                            paths = caminho_arquivo.split('/')
                            file_csv = paths[-1]
                            estado = file_csv.split('_')[2]
                            derivado = file_csv.split('_')[3].replace('.csv', '')
                            cidade = file_csv.split('_')[1]
                            df = pd.read_csv(caminho_arquivo, sep=";")
                            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y')

                            df.set_index('timestamp', inplace=True)
                            start_timestamp = '1998-01-01'  # ou qualquer data inicial que você queira
                            index_series = pd.date_range(start=start_timestamp, periods=len(df), freq='Y')
                            # series = pd.Series(series_value, index=index_series)
                            serie_m3 = pd.Series(df['m3'].values, index=index_series)
                            train, test = train_test_stats(serie_m3, horizon)
                            w = len(train)
                            # train_tf, _, _ = rolling_window_series(train, horizon)
                            train_tf = znorm(train)
                            train_darts = TimeSeries.from_series(train_tf)
                            
                            if len(serie_m3) > 12:
                                test_index = test
                                #roda 5 vezes para 5 anos
                                preds_final = []
                                preds_series = pd.Series()
                                train_concat = train
                                # model = StatsForecastAutoETS(model='AAN')
                                # model = StatsForecastAutoARIMA(season_length=1)
                                model = ETSModel(train_tf, trend='add', damped_trend=True, seasonal_periods=1, freq='A').fit(full_output=False, disp=False)


                                predictions = model.forecast(steps=horizon)
                                preds_norm = pd.Series(predictions, index=test.index)
                                # model.fit(train_darts)
                                # result = model.predict(n=horizon)
                                # preds_norm = pd.Series(result.values().flatten().tolist(), index=test.index)
                                preds_real = reverse_transform_norm_preds(preds_norm, train_concat, "normal", w=horizon)
                                preds_final = preds_real.values
                                # for i in range(5):
                                #     model = StatsForecastAutoETS()
                                #     model.fit(train_darts)
                                #     result = model.predict(n=1)
                                #     new_index = test_index.index[0] + pd.DateOffset(years=i)
                                #     new_index = pd.DatetimeIndex([new_index])
                                #     preds_norm = pd.Series(result.values().flatten().tolist(), index=new_index)
                                #     w = len(train_concat)
                                #     preds_real = reverse_transform_norm_preds(preds_norm, train_concat, "normal", w=w)
                                #     train_concat = pd.concat([train_concat, preds_real])
                                #     preds_final.extend(preds_real.values)
                                #     if preds_series.empty:
                                #         preds_series = preds_real
                                #     else:
                                #         preds_series = pd.concat([preds_series, preds_real])
                                    
                                #     #concat train com preds_real
                                #     # train_concat_tf, _, _ = rolling_window_series(train_concat, w)
                                #     train_concat_tf = znorm(train_concat)
                                #     train_darts = TimeSeries.from_series(train_concat_tf)
                                    
                                cont+=1
                                mape_result = mape(test.values, preds_final)
                                pocid_result = pocid(test.values, preds_final)
                                dados = {
                                    'PRODUCT': derivado,
                                    'UF': estado.upper(),
                                    'MODEL': 'ETS',
                                    'PARAMETERS': str({'trend':'add', 'damped_trend':True, 'seasonal_periods':1, 'freq':'A'}),
                                    # 'mape': mape_result,
                                    # 'pocid': pocid_result,
                                    'city': cidade,
                                    'test': [list(test.values)],
                                    'preds': [list(preds_final)],
                                    # 'index_test': [test.index],
                                    # 'index_preds': [preds_series.index]
                                    
                                }
                                df_result = pd.DataFrame(dados)
                                df_result = df_result.reset_index(drop=True)
                                print(cont)

                                if not os.path.exists(resultado_file):
                                    df_result.to_csv(resultado_file, sep=';', index=False, mode='w', header=True)
                                else:
                                    df_result.to_csv(resultado_file, sep=';', index=False, mode='a', header=False)
                        except Exception as e:
                            print(f"Erro ao ler {arquivo}: {e}")
        else:
            print(train)
            print(f"O diretório {diretorio} não existe ou não é uma pasta válida.")




/home/user/miniconda3/envs/lucas/lib/python3.11/site-packages/optuna/study/_optimize.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [49]:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:.4f}".format

model = "FT_catboost"
part = '/normal'
csv_files = [
    f"./timeseries/mestrado/resultados/{model}{part}/ANP_MONTHLY.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/M4_HOURLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/M4_WEEKLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/NN5_WEEKLY_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/PEDESTRIAN_COUNTS_DATASET.csv",
    f"./timeseries/mestrado/resultados/{model}{part}/US_BIRTHS_DATASET.csv",
]

for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, sep=";")
        print(f"{csv_file.split('/')[-1]}: {len(df)} entries")
    except Exception as e:
        print(f"Erro ao ler {csv_file}: {e}")

ANP_MONTHLY.csv: 5642 entries
AUSTRALIAN_ELECTRICITY_DEMAND_DATASET.csv: 20 entries
M4_HOURLY_DATASET.csv: 1656 entries
M4_WEEKLY_DATASET.csv: 1436 entries
NN5_DAILY_DATASET_WITHOUT_MISSING_VALUES.csv: 444 entries
NN5_WEEKLY_DATASET.csv: 444 entries
PEDESTRIAN_COUNTS_DATASET.csv: 264 entries
US_BIRTHS_DATASET.csv: 4 entries


In [13]:
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:.4f}".format
df_agent = pd.read_csv(
    "./timeseries/mestrado/resultados/agent_qwen-3-coder-480b/ANP_MONTHLY.csv",
    sep=";",
)
# pegar cada valor das colunas mape, pocid, smape, rmse, msmape, mae e gerar uma media de todos
print(
    df_agent[["mape", "pocid", "smape", "rmse", "msmape", "mae"]]
    .apply(pd.to_numeric, errors="coerce")
    .mean()
)

mape     1623941381919985.0000
pocid                  60.0305
smape                   0.2337
rmse                 7210.6713
msmape                  0.2336
mae                  6251.2619
dtype: float64
